In [1]:
import pandas as pd
import ast

In [37]:
emails = pd.read_csv('gs://user-scripts-msca310019-capstone-49b3/data/data_message_reply_pairs_cleaned.csv')

In [13]:
len(emails)

24640

In [40]:
emails.columns

Index(['Unnamed: 0', 'time', 'subject', 'thread', 'sender', 'recipient',
       'message', 'reply_time', 'reply_sender', 'reply_recipient',
       'reply_message'],
      dtype='object')

In [41]:
emails = emails.rename(columns = {'Unnamed: 0':'Email_ID'})

In [42]:
emails['recipient'] = emails['recipient'].apply(ast.literal_eval)

In [17]:
emails.head(13)

,Email_ID,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message
0,0,1999-06-09 04:18:00-07:00,RE: test,2,5552,[40034],How about this Friday ? Julie has not left yet...,1999-06-09 08:06:00-07:00,40034,[5552],when? how are you and your family? is julie go...
1,1,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie go...,1999-06-10 03:54:00-07:00,5552,[40034],Today is bad. Tommorrow I will call you.
2,2,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie go...,1999-11-23 01:38:00-08:00,5552,[40034],Do you have lunch plans today?
3,3,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie go...,1999-11-23 03:13:00-08:00,5552,[40034],Really? I'd feel like a mooch. Lets have lunch...
4,4,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie go...,1999-11-23 03:58:00-08:00,5552,[40034],Tues.is good. I'll call you.
5,7,2001-09-07 07:00:56-07:00,RE: test,2,1033,[12830],"Hi, I got your email. Thank you for keeping me...",2001-09-07 14:12:38-07:00,12830,[1033],Great! I'll look forward to meeting you. Candi...
6,8,2001-10-19 09:10:09-07:00,test,2,14587,[32780],"Larry, I'm checking your email so if you get t...",2001-10-19 09:15:37-07:00,32780,[14587],Your test was successful How's it going?
7,9,2001-10-19 09:15:37-07:00,RE: test,2,32780,[14587],Your test was successful How's it going?,2001-10-19 10:00:58-07:00,14587,"[32795, 32780]","We are meeting for drinks after work today, so..."
8,10,2001-10-19 10:00:58-07:00,RE: test,2,14587,"[32795, 32780]","We are meeting for drinks after work today, so...",2001-10-19 10:05:47-07:00,32780,[14587],I'm going to Bob's Halloween party on the 27th...
9,11,2001-10-19 10:05:47-07:00,RE: test,2,32780,[14587],I'm going to Bob's Halloween party on the 27th...,2001-10-19 10:20:46-07:00,14587,"[32795, 32780]",We are thinking of renting a Limo Saturday nig...


In [18]:
sender_df = emails[emails.sender == 39298]
sender_df.head()

,Email_ID,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message
10,13,2001-11-08 14:18:15-08:00,test,2,39298,[402],test Dave,2001-11-09 09:03:31-08:00,402,[39298],"Dave, Take a look at this before you pass it o..."


In [19]:
len(sender_df)

1

In [20]:
top_emails = sender_df[sender_df['recipient'].apply(lambda x: any(item in x for item in [1721]))]

In [21]:
top_emails.head()

,Email_ID,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message


In [22]:
len(top_emails)

0

In [51]:
def get_sender_receiver_emails(row):
    sender = row['sender']
    recipient = row['reply_sender']
    email_id = row['Email_ID']
        
    recipient_df = emails[emails.reply_sender == recipient]
    recipient_df = recipient_df[recipient_df.Email_ID != email_id]
    
    top_emails = recipient_df[recipient_df['sender']==sender]    
    
    if(top_emails.empty):
        return ""
    
    relevent_emails = list(top_emails['reply_message'])
    relevent_emails = [str(item) for item in relevent_emails]
    
#     print(len(relevent_emails))
#     print('\n')
#     combined_string = " ".join(relevent_emails)
#     return combined_string
    return relevent_emails

def get_sender_emails(row):
    sender = row['reply_sender']
    email_id = row['Email_ID']
        
    sender_df = emails[emails.reply_sender == sender]
    sender_df = sender_df[sender_df.Email_ID != email_id]

    if(sender_df.empty):
        return ""
    
    relevent_emails = list(sender_df['reply_message'])
    relevent_emails = [str(item) for item in relevent_emails]
    
#     print(len(relevent_emails))
#     print('\n')
    
#     combined_string = " ".join(relevent_emails)
    
#     return combined_string
    return relevent_emails

def remove_same_emails(row):
    all_emails = row['Sender_Emails_All']
    sender_receiver_emails = row['Sender_Receiver_Emails']    
    
    return [x for x in all_emails if x not in sender_receiver_emails]

In [52]:
emails['Sender_Receiver_Emails'] = emails.apply(get_sender_receiver_emails,axis=1)
# emails['Sender_Emails_All'] = emails.apply(get_sender_emails,axis=1)

# emails['Sender_Emails_Filtered'] = emails.apply(remove_same_emails,axis=1)

In [53]:
emails.head()

,Email_ID,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message,Sender_Receiver_Emails
0,0,1999-06-09 04:18:00-07:00,RE: test,2,5552,[40034],How about this Friday ? Julie has not left yet...,1999-06-09 08:06:00-07:00,40034,[5552],when? how are you and your family? is julie go...,
1,1,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie go...,1999-06-10 03:54:00-07:00,5552,[40034],Today is bad. Tommorrow I will call you.,"[Do you have lunch plans today?, Really? I'd f..."
2,2,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie go...,1999-11-23 01:38:00-08:00,5552,[40034],Do you have lunch plans today?,"[Today is bad. Tommorrow I will call you., Rea..."
3,3,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie go...,1999-11-23 03:13:00-08:00,5552,[40034],Really? I'd feel like a mooch. Lets have lunch...,"[Today is bad. Tommorrow I will call you., Do ..."
4,4,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie go...,1999-11-23 03:58:00-08:00,5552,[40034],Tues.is good. I'll call you.,"[Today is bad. Tommorrow I will call you., Do ..."


In [ ]:
emails_small = emails.drop(columns = ['Sender_Emails_All','Sender_Emails_Filtered'])

In [ ]:
emails_small.to_csv('gs://user-scripts-msca310019-capstone-49b3/data/20231008_Emails_with_Previous_as_List.csv')

emails_small.to_csv('gs://user-scripts-msca310019-capstone-49b3/data/20231019_Emails_with_Sender_Reply.csv')


In [20]:
emails = emails.drop(columns=['Sender_Emails_All'])

In [21]:
emails.to_csv('gs://user-scripts-msca310019-capstone-49b3/data/20231008_Emails_with_All_Previous.csv')

In [ ]:
def email_retreival(sender_id,recipients,df):
    sender_df = df[df.sender == sender_id]
    print(sender_df)
            